IBM Data Science Capstone course.
This notebook is for part 2 of week 3 assignment.
I copied the code of the first part of the assignment to create the dataframe and removed markdowns and comments. For the complete code with markdowns, please see "capstone_w3_hw1" in the same repository.

In [17]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

# get wikipedia page html
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source,'lxml')

#after inspecting html of page, the class of the table is found
table = soup.find('table', {"class": "wikitable sortable"})

rows=table.find_all('tr') #all table's rows
# find name of columns from first row of table
col_hs=rows[0].find_all('th') #columns name have tag <th
col_names=[colh.text.rstrip('\n') for colh in col_hs]
table_data=[]
for row in rows[1:]:
    col_data=row.find_all('td') #table elements have tag <td
    col_data=[col.text.rstrip('\n') for col in col_data]
    table_data.append([cell_data for cell_data in col_data])
table_df = pd.DataFrame(table_data, columns=col_names) #create dataframe

# Remove rows that don't have an assigned borough
table_df1=table_df[table_df['Borough']!='Not assigned'].reset_index().drop(['index'],axis=1)

# Assign borough's name to neighborhoods that don't have a name
table_df1['Neighbourhood'][table_df1['Neighbourhood']=='Not assigned']=table_df1['Borough']

# list of unique post codes and corresponding boroughs
df2=table_df1.drop_duplicates(subset=['Postcode','Borough']).sort_values(by='Postcode',axis=0,ascending=True)
postc_list=df2['Postcode']
boro_list=df2['Borough']

neigh_list=[]
s=', ' #delimiter
for pc in postc_list:
    neigh_list.append(s.join(table_df1[table_df1['Postcode']==pc]['Neighbourhood']))

newdf=pd.DataFrame({'PostalCode':postc_list, 'Borough': boro_list,'Neighborhood': neigh_list}).reset_index(drop=True)
print('The new dataframe has {} rows'.format(newdf.shape[0]))

The new dataframe has 103 rows


The second part of this week assignment starts here. Due to issues in using the Python Geocoder package, I used the provided csv file with latitudes and longitudes of Toronto's postal codes. It's downloaded in my local directory. Since the dataframe I created in the previous step is sorted in ascending order, there's no need to re-sort, it already matches the order of the latitude,longitude in the csv file. 

In [20]:
# dataframe of coordinates
coord_df=pd.read_csv("Geospatial_Coordinates.csv")

newdf[['Latitude','Longitude']]=coord_df[['Latitude','Longitude']]
newdf.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
